## HW2 of CSE 6740
## Programming: Text Clustering
In this problem, we will explore the use of EM algorithm for text clustering. Text clustering is a technique for unsupervised document organization, information retrieval. We want to find how to group a set of different text documents based on their topics. First we will analyze a model to represent the data.

### Bag of Words
The simplest model for text documents is to understand them as a collection of words. To keep the model simple, we keep the collection unordered, disregarding grammar and word order. What we do is counting how often each word appears in each document and store the word counts into a matrix, where each row of the matrix represents one document. Each column of matrix represent a specific word from the document dictionary. Suppose we represent the set of nd documents using a matrix of word counts like this:

$$
D_{1: n_d}=\left(\begin{array}{cccc}
2 & 6 & \ldots & 4 \\
2 & 4 & \ldots & 0 \\
\vdots & & \ddots &
\end{array}\right)=T
$$


This means that word $W_1$ occurs twice in document $D_1$. Word $W_{n_w}$ occurs 4 times in document $D_1$ and not at all in document $D_2$.

### Multinomial Distribution
The simplest distribution representing a text document is multinomial distribution (Bishop Chapter 2.2). The probability of a document $D_i$ is:

$$
p\left(D_i\right)=\prod_{j=1}^{n_w} \mu_j^{T_{i j}}
$$


Here, $\mu_j$ denotes the probability of a particular word in the text being equal to $w_j, T_{i j}$ is the count of the word in document. So the probability of document $D_1$ would be $p\left(D_1\right)=\mu_1^2 \cdot \mu_2^6 \cdot \ldots \cdot \mu_{n_w}^4$.

### Mixture of Multinomial Distributions
In order to do text clustering, we want to use a mixture of multinomial distributions, so that each topic has a particular multinomial distribution associated with it, and each document is a mixture of different topics. We define $p(c)=\pi_c$ as the mixture coefficient of a document containing topic $c$, and each topic is modeled by a multinomial distribution $p\left(D_i \mid c\right)$ with parameters $\mu_{j c}$, then we can write each document as a mixture over topics as

$$
p\left(D_i\right)=\sum_{c=1}^{n_c} p\left(D_i \mid c\right) p(c)=\sum_{c=1}^{n_c} \pi_c \prod_{j=1}^{n_w} \mu_{j c}^{T_{i j}}
$$


### EM for Mixture of Multinomials
In order to cluster a set of documents, we need to fit this mixture model to data. In this problem, the EM algorithm can be used for fitting mixture models. This will be a simple topic model for documents. Each topic is a multinomial distribution over words (a mixture component). EM algorithm for such a topic model, which consists of iterating the following steps:

**Expectation**

Compute the expectation of document $D_i$ belonging to cluster $c$ :

$$
\gamma_{i c}=\frac{\pi_c \prod_{j=1}^{n_w} \mu_{j c}^{T_{i j}}}{\sum_{c=1}^{n_c} \pi_c \prod_{j=1}^{n_w} \mu_{j c}^{T_{i j}}}
$$

**Maximization**

Update the mixture parameters, i.e. the probability of a word being $W_j$ in cluster (topic) $c$, as well as prior probability of each cluster.

$$
\begin{gathered}
\mu_{j c}=\frac{\sum_{i=1}^{n_d} \gamma_{i c} T_{i j}}{\sum_{i=1}^{n_d} \sum_{l=1}^{m_w} \gamma_{i c} T_{i l}} \\
\pi_c=\frac{1}{n_d} \sum_{i=1}^{n_d} \gamma_{i c}
\end{gathered}
$$


### Task

Implement the algorithm and run on the toy dataset `data.mat`. Observe the results and compare them with the provided true clusters each document belongs to. Report the evaluation (e.g. accuracy) of your implementation.
Hint: We already did the word counting for you, so the data file only contains a count matrix like the one shown above. For the toy dataset, set the number of clusters $n_c$ = 4. You will need to initialize the parameters. Try several different random initial values for the probability of a word being $W_j$ in topic c, $μ_{jc}$. Make sure you normalized it. Make sure that you should not use the true cluster information during your learning phase.

In [48]:
import scipy.io
import itertools
import numpy as np

def acc_measure(idx, Y):
    """
    Function for evaluate the accuracy
    :param idx:
        numpy array of (num_doc)
    :param Y:
        numpy array of (num_doc)
    :return:
        accuracy
    """
    # rotate for different idx assignments
    best_acc = 0
    for idx_order in itertools.permutations([1, 2, 3, 4]):

        for ind, label in enumerate(idx_order):
            Y[(ind)*100:(ind+1)*100] = label

        acc = (Y == idx).sum() / Y.shape[0]
        if acc > best_acc:
            best_acc = acc

    return best_acc

# Example:
#     acc_measure(np.array([1]*100 + [3]*100 + [2]*100 + [4]*100))

def cluster(bow, K, num_iters = 1000, epsilon = 1e-12):
    """
    # TODO: Finish the cluster function
    :param bow:
        bag-of-word matrix of (num_doc, V), where V is the vocabulary size
    :param K:
        number of topics
    :return:
        idx of size (num_doc), idx should be 1, 2, 3 or 4
    """
    
    
    num_docs, num_words = bow.shape
        
    # Initialize parameters
    pi = np.random.rand(K)
    pi /= np.sum(pi) 

    mu = np.random.rand(K, num_words)
    mu /= mu.sum(axis=1, keepdims=True)  


    for _ in range(num_iters):
        # E-Step
        gamma = pi * np.prod(mu ** bow[:, np.newaxis, :], axis=2)  # Shape: (num_docs, K)
        gamma /= gamma.sum(axis=1, keepdims=True)  # Normalize 
        old_pi = pi.copy()
        old_mu = mu.copy()
        # M-Step: Update pi and mu 
        pi = gamma.sum(axis=0) / num_docs  # Shape: (K,)

        mu = (gamma.T @ bow)  # Shape: (K, num_words)
        mu /= mu.sum(axis=1, keepdims=True)  # Normalize across words for each topic
        #check for convergence
        if np.linalg.norm(pi - old_pi) < epsilon and np.linalg.norm(mu - old_mu) < epsilon:
            print('Converged')
            break
        


    # Assign each document
    idx = np.argmax(gamma, axis=1) + 1  # Add 1 to make the labels 1-based
    return idx

# Evaluation
mat = scipy.io.loadmat('data.mat')
mat = mat['X']
X = mat[:, :-1]
Y = mat[:, -1]
for i in range (4):
    idx = cluster(X, 4)
    acc = acc_measure(idx, Y)
    print(f'{i} : accuracy %.4f' % (acc))

Converged
0 : accuracy 0.8750
Converged
1 : accuracy 0.8150
Converged
2 : accuracy 0.8025
Converged
3 : accuracy 0.8200


## Extra Credit: Realistic Topic Models
The above model assumes all the words in a document belongs to some topic at the same time. However, in real world datasets, it is more likely that some words in the documents belong to one topic while other words belong to some other topics. For example, in a news report, some words may talk about “Ebola” and “health”, while others may mention “administration” and “congress”. In order to model this phenomenon, we should model each word as a mixture of possible topics. 

Specifically, consider the log-likelihood of the joint distribution of document and words


$$
\mathcal{L}=\sum_{d \in \mathcal{D}} \sum_{w \in \mathcal{W}} T_{d w} \log P(d, w)
$$

where $T_{d w}$ is the counts of word $w$ in the document $d$. This count matrix is provided as input.
The joint distribution of a specific document and a specific word is modeled as a mixture

$$
P(d, w)=\sum_{z \in \mathcal{Z}} P(z) P(w \mid z) P(d \mid z)
$$

where $P(z)$ is the mixture proportion, $P(w \mid z)$ is the distribution over the vocabulary for the $z$-th topic, and $P(d \mid z)$ is the probability of the document for the $z$-th topic. And these are the parameters for the model.

The E-step calculates the posterior distribution of the latent variable conditioned on all other variables

$$
P(z \mid d, w)=\frac{P(z) P(w \mid z) P(d \mid z)}{\sum_{z^{\prime}} P\left(z^{\prime}\right) P\left(w \mid z^{\prime}\right) P\left(d \mid z^{\prime}\right)}
$$


In the M-step, we maximizes the expected complete log-likelihood with respect to the parameters, and get the following update rules

$$
\begin{aligned}
P(w \mid z) & =\frac{\sum_d T_{d w} P(z \mid d, w)}{\sum_{w^{\prime}} \sum_d T_{d w^{\prime}} P\left(z \mid d, w^{\prime}\right)} \\
P(d \mid z) & =\frac{\sum_w T_{d w} P(z \mid d, w)}{\sum_{d^{\prime}} \sum_w T_{d^{\prime} w} P\left(z \mid d^{\prime}, w\right)} \\
P(z) & =\frac{\sum_d \sum_w T_{d w} P(z \mid d, w)}{\sum_{z^{\prime}} \sum_{d^{\prime}} \sum_{w^{\prime}} T_{d^{\prime} w^{\prime}} P\left(z^{\prime} \mid d^{\prime}, w^{\prime}\right)}
\end{aligned}
$$

## Task
Implement EM for maximum likelihood estimation and cluster the text data provided in the nips.mat file you downloaded. You can print out the top key words for the topics/clusters by using the show `display_topics` utility. It takes two parameters: 1) your learned conditional distribution matrix, i.e., $P(w|z)$ and 2) a cell array of words that corresponds to the vocabulary. You can find the cell array wl in the nips.mat file. Try different values of $k$ and see which values produce sensible topics. In assessing your code, we will use another dataset and observe the produces topics.

In [14]:
import scipy.io
import itertools
import numpy as np

def cluster_extra(T, K, num_iters = 50, epsilon = 1e-12):
    """
    TODO: Finish the function of clustering.
    :param bow:
        bag-of-word matrix of (num_doc, V), where V is the vocabulary size
    :param K:
        number of topics
    :return:
        word-topic matrix of (V, K)
    """
    T=T.toarray()
    num_docs, vocab_size = T.shape
    Word_topic_matrix = np.random.rand(vocab_size, K)  # P(w|z)
    Doc_topic_probs = np.random.rand(num_docs, K)   # P(d|z)
    Topic_probabilities = np.random.rand(K)  # P(z)

    # Normalize the initial distributions
    Word_topic_matrix /= Word_topic_matrix.sum(axis=0)  
    Doc_topic_probs /= Doc_topic_probs.sum(axis=0)  
    Topic_probabilities /= Topic_probabilities.sum()        

    # EM loop
    for iteration in range(num_iters):
        
        # E-Step: Compute responsibilities P(z | d, w)

        Responsibility_matrix = Topic_probabilities * Doc_topic_probs[:, np.newaxis, :] * Word_topic_matrix[np.newaxis, :, :]
        Responsibility_matrix /= Responsibility_matrix.sum(axis=2, keepdims=True)  # Normalize
        
        # M-Step
        matrix = (T[:, :, np.newaxis] * Responsibility_matrix)
        # Update P(w|z)
        New_word_topic_matrix= matrix.sum(axis=0)
        New_word_topic_matrix /= New_word_topic_matrix.sum(axis=0)
        # Update P(d|z)
        New_doc_topic_probs = matrix.sum(axis=1)
        New_doc_topic_probs /= New_doc_topic_probs.sum(axis=0)
        # Update P(z)
        New_topic_prob = matrix.sum(axis=(0, 1))
        New_topic_prob /= New_topic_prob.sum()

        # Check for convergence
        x= np.linalg.norm(New_topic_prob-Topic_probabilities)
        if x < 1e-4:
            print("Converged")
            break

        # Update parameters
        Word_topic_matrix, Doc_topic_probs, Topic_probabilities = New_word_topic_matrix, New_doc_topic_probs, New_topic_prob

    return Word_topic_matrix
    
   

def display_topics(W, wl):
    """

    :param W:
        word-topic matrix of size (V, K)
    :param wl:
        array of str of size (V)
    :return:
    """
    top_n_words = 10
    ind_mat = np.argsort(-W.T, axis=1)[:, :top_n_words]
    for k_ind in range(ind_mat.shape[0]):
        w_ls = [wl[ind, 0][0] for ind in ind_mat[k_ind]]
        print('topic %i: %s' % (k_ind, ' '.join(w_ls)))
        

## Evaluation for displaying topics
n_topics = 5 # TODO specify num topics yourself
cell = scipy.io.loadmat('nips.mat')

mat = cell['raw_count'] # sparse mat of size (num_doc, num_words)
wl = cell['wl']

for n_topics in range(2,6):
    print(f'\n Number of topics: {n_topics}')
    W = cluster_extra(mat, n_topics)
    display_topics(W, wl)


 Number of topics: 2
topic 0: learning model input neural networks data network figure set time
topic 1: network output training neural point parameters error examples function learning

 Number of topics: 3
topic 0: model neural time training set figure data learning output function
topic 1: learning set network input algorithm neural time system number data
topic 2: network model learning figure input function output networks results data

 Number of topics: 4
topic 0: learning neural training algorithm data function model based system order
topic 1: model input network set time figure output networks training state
topic 2: network information time neural set learning input networks hidden model
topic 3: learning network neural function figure data error model system networks

 Number of topics: 5
topic 0: network input model learning data output figure training state set
topic 1: networks neural model function network time number figure input learning
topic 2: network learning neu